In [1]:
from hmoe_buildkit import HMOEBuilder
model_names = [
    "HuggingFaceTB/SmolVLM-500M-Instruct",
    "meta-llama/Llama-3.2-1B",
    "Qwen/Qwen3-0.6B"
]
builder = HMOEBuilder(model_names, 2)

In [2]:
tokenizer, model = builder.build()

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [3]:
print(model)

HMOE(
  (vision_encoder): Idefics3Model(
    (vision_model): Idefics3VisionTransformer(
      (embeddings): Idefics3VisionEmbeddings(
        (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
        (position_embedding): Embedding(1024, 768)
      )
      (encoder): Idefics3Encoder(
        (layers): ModuleList(
          (0-11): 12 x Idefics3EncoderLayer(
            (self_attn): Idefics3VisionAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
            (mlp): Idefics3VisionMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features=768, out_features=30

In [4]:
from transformers.image_utils import load_image
image = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Can you describe this image?"}
        ]
    },
]

# Prepare inputs
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
inputs = tokenizer(text=prompt, images=[image], return_tensors="pt")

In [5]:
import torch
inputs = inputs.to('cuda')
model = model.to('cuda')
with torch.no_grad():
    outputs = model(**inputs)
outputs

CausalLMOutputWithPast(loss=None, logits=tensor([[[ -0.2061,  -5.3438,  -7.4062,  ...,  -1.7031,   0.3535,   0.8047],
         [ -3.0781,  -7.5000, -14.3125,  ...,  -4.2188,   0.4863,   1.6875],
         [  4.9062,  -7.8125,  -7.3438,  ...,   2.7969,   1.6328,   2.4688],
         ...,
         [  3.5625,   1.6953,   3.8438,  ...,   3.1875,  -1.2266,   2.3125],
         [ -1.0938,  -0.1973,   3.4531,  ...,   2.3750,   0.8359,   0.9297],
         [ -0.3984,  -6.8438,   1.6406,  ...,   0.6680,  -1.6172,  -1.0859]]],
       device='cuda:0', dtype=torch.bfloat16), past_key_values=None, hidden_states=None, attentions=None)

In [6]:
with torch.no_grad():
    outputs = model.generate(**inputs)
tokenizer.batch_decode(outputs)

['<|im_start|>User:<fake_token_around_image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><fake_token_around_image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><fake_token_around_im

In [7]:
tokenizer.save_pretrained('outputs/hmoe')
model.save_pretrained('outputs/hmoe')

[2025-07-16 22:26:54,139] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/coder/miniconda3/envs/default/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/coder/miniconda3/envs/default/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


[2025-07-16 22:26:55,509] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


In [1]:
# import hmoe_buildkit
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('outputs/pretraining')
model = AutoModelForCausalLM.from_pretrained(
    'outputs/pretraining', 
    torch_dtype='bfloat16', 
    trust_remote_code=True
)
model

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


HMOE(
  (vision_encoder): Idefics3Model(
    (vision_model): Idefics3VisionTransformer(
      (embeddings): Idefics3VisionEmbeddings(
        (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
        (position_embedding): Embedding(1024, 768)
      )
      (encoder): Idefics3Encoder(
        (layers): ModuleList(
          (0-11): 12 x Idefics3EncoderLayer(
            (self_attn): Idefics3VisionAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
            (mlp): Idefics3VisionMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features=768, out_features=30